# Semantické jádro

V tomto příkladu kódu použijete AI Framework [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) k vytvoření základního agenta.

Cílem tohoto příkladu je ukázat vám kroky, které později použijeme v dalších příkladech kódu při implementaci různých agentních vzorců.


## Importujte potřebné balíčky Pythonu


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Vytvoření klienta

V tomto příkladu použijeme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pro přístup k LLM.

`ai_model_id` je definován jako `gpt-4o-mini`. Zkuste změnit model na jiný dostupný na tržišti GitHub Models a podívejte se na různé výsledky.

Abychom mohli použít `Azure Inference SDK`, který se používá pro `base_url` u GitHub Models, použijeme konektor `OpenAIChatCompletion` v rámci Semantic Kernel. Existují také další [dostupné konektory](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), které umožňují použití Semantic Kernel pro jiné poskytovatele modelů.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Vytvoření agenta

Níže vytvoříme agenta s názvem `TravelAgent`.

Pro tento příklad používáme velmi jednoduché pokyny. Můžete tyto pokyny změnit, abyste viděli, jak agent reaguje odlišně.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Spuštění agenta

Nyní můžeme spustit agenta definováním vlákna typu `ChatHistoryAgentThread`. Jakékoli potřebné systémové zprávy jsou předány agentovi prostřednictvím klíčového argumentu `messages` funkce invoke_stream.

Po jejich definování vytvoříme `user_inputs`, což bude zpráva, kterou uživatel posílá agentovi. V tomto případě jsme nastavili tuto zprávu na `Naplánuj mi slunečnou dovolenou`.

Neváhejte tuto zprávu změnit a zjistit, jak agent reaguje odlišně.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
